In [1]:
import numpy as np
from ReadBinary import *

def GetViewArrays(view, simfolder="../..", max_level=-1):
    arrList = []
    meshList = []
    axisRange = [[], []]
    axisLabel = []
    levels = []
    maxPoint_center = None

    def AdjustAxisMinMax(axisRange, new_min_max):
        ax_min, ax_max = new_min_max
        if axisRange == []:
            axisRange.extend([ax_min, ax_max])
        else:
            if axisRange[0] > ax_min:
                axisRange[0] = ax_min
            if axisRange[1] < ax_max:
                axisRange[1] = ax_max

    if view["type"] == "partial":
        for gridName, params in gridParams.items():
            if max_level > 0 and params["level"] > max_level:
                continue
            r0, r1 = params["r0"], params["r1"]
            plane = {"x":0, "y":1, "z":2}
            if r0[plane[view["plane"]]] <= view["at"] < r1[plane[view["plane"]]]:
                dataFileName =  simfolder + "/build/data/3D/auto/" + \
                                gridName + "_" + view["arrayName"] + "_" + view["direction"] + \
                                "_@" + view["plane"] + "=" + str(view["at"]) + ".data"
                arr = GetArrays(dataFileName)
                H, V = None, None
                hRange, vRange = axisRange
                axLabel = None
                level = params["level"]
                if level == 0:
                    ind_max = np.unravel_index(np.argmax(arr, axis=None), arr.shape)
                    maxPoint_center = [ind_max, arr[ind_max]]
                    print("maxPoint_center: ", maxPoint_center)
                if view["plane"] == "x":
                    arr = arr[:, 0, :, :]
                    z = np.linspace(r0[2], r1[2], arr.shape[2])
                    y = np.linspace(r0[1], r1[1], arr.shape[1])
                    Z, Y = np.meshgrid(z, y, indexing="xy")
                    H, V = Z, Y
                    AdjustAxisMinMax(hRange, (r0[2], r1[2]))
                    AdjustAxisMinMax(vRange, (r0[1], r1[1]))
                    axLabel = ["z", "y"]
                        
                elif view["plane"] == "y":
                    arr = arr[:, :, 0, :]
                    z = np.linspace(r0[2], r1[2], arr.shape[2])
                    x = np.linspace(r0[0], r1[0], arr.shape[1])
                    Z, X = np.meshgrid(z, x, indexing="xy")
                    H, V = Z, X
                    AdjustAxisMinMax(hRange, (r0[2], r1[2]))
                    AdjustAxisMinMax(vRange, (r0[0], r1[0]))
                    axLabel = ["z", "x"]

                elif view["plane"] == "z":
                    arr = arr[:, :, :, 0]
                    y = np.linspace(r0[1], r1[1], arr.shape[2])
                    x = np.linspace(r0[0], r1[0], arr.shape[1])
                    X, Y = np.meshgrid(x, y, indexing="ij")
                    H, V = X, Y
                    AdjustAxisMinMax(hRange, (r0[0], r1[0]))
                    AdjustAxisMinMax(vRange, (r0[1], r1[1]))
                    axLabel = ["x", "y"]
                    
                else:
                    assert False
                arrList.append(arr)
                meshList.append([H, V])
                axisRange = [hRange, vRange]
                axisLabel = axLabel
                levels.append(level)
                
    return arrList, meshList, axisRange, axisLabel, levels, maxPoint_center


import pickle
fileParams = open("../data/3D/auto/params.param", "rb")
params = pickle.load(fileParams)
fileParams.close()
#print("params: ", params)

gridParams = params["grids"]
viewParams = params["views"]
unitParams = params["units"]
boxesParams = params["boxes"]
sourceParams = params["sources"][0]
geomParams = params["geometries"][0]

dt_coarse = boxesParams[-1]["dt"]
length_FD = unitParams["length_FD"]

from PhysicalUnits import PhysicalUnits
units = PhysicalUnits(length_FD)
dt_SI = units.ConvertFDTimeToSIUnits(dt_coarse)

tipHeight = geomParams["height"]

print("views: ")
for i in range(len(viewParams)):
    print(i, " : ", viewParams[i])

print(sourceParams)
A_src = sourceParams["amplitude"]
v_src = sourceParams["propagationVelocity"]
t0_src = sourceParams["t_center"]
w_src = 2.0*np.pi*sourceParams["modulationFrequency"]
phi_src = sourceParams["modulationPhase"]
t_width_src = sourceParams["rectWidth"]
t_edgewidth_src = sourceParams["rectEdgeWidth"]
def rect(x, width, edge_width):
    return (np.abs(x) <= (width - edge_width)/2) \
           + 0.5 * (np.abs(x + width/2) < edge_width/2) * (1.0 + np.sin((x + width/2)/edge_width * np.pi)) \
           + 0.5 * (np.abs(x - width/2) < edge_width/2) * (1.0 - np.sin((x - width/2)/edge_width * np.pi))

def GetSourceField(t, plane, H, V):
    if plane == "x":
        Ey_inc = A_src*np.cos(w_src*(t - t0_src - H/v_src) + phi_src) \
                 * rect(t - t0_src - H/v_src, t_width_src, t_edgewidth_src)
        return Ey_inc
    elif plane == "y":
        Ey_inc = A_src*np.cos(w_src*(t - t0_src - H/v_src) + phi_src) \
                 * rect(t - t0_src - H/v_src, t_width_src, t_edgewidth_src)
        return Ey_inc
    elif plane == "z":
        Ey_inc = A_src*np.cos(w_src*(t - t0_src) + phi_src) \
                 * rect(t - t0_src, t_width_src, t_edgewidth_src)
        return Ey_inc

#plot(GetSourceField(t0_src, "x", np.linspace(-200, +200, 10000), 0.0))
#plot(rect(np.linspace(-2, 2, 100) - 0.5, 2, 0.5))

views: 
0  :  {'type': 'partial', 'plane': 'x', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
1  :  {'type': 'partial', 'plane': 'y', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
2  :  {'type': 'partial', 'plane': 'z', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
{'type': 'PureScatteredRectPlaneWave', 'geometryName': 'tip', 'polarization': 'y', 'amplitude': 1.0, 'propagationDirection': array([0., 0., 1.]), 'propagationVelocity': 1.0, 't_center': 17.987547479999996, 'rectWidth': 29.979245799999998, 'rectEdgeWidth': 2.99792458, 'modulationFrequency': 0.03335640951981521, 'modulationPhase': 1.5707963267948966}


In [3]:
%pylab tk
import numpy as np
from scipy import constants
from matplotlib import pyplot as plt
import matplotlib.animation as animation

viewInd = 0
arrList, meshList, axisRange, axisLabel, levels, maxPoint_center = GetViewArrays(viewParams[viewInd])
plot_mag = False
lengthScale_si = length_FD/constants.micro
lengthUnit = "um"
timeScale_si = dt_SI/constants.pico
timeUnit = "ps"
showTotalField = True
showIncidentField = False
normalizeToSource = True
zoom = True
zoom_level = 0

if zoom:
    arrList = [arrList[i] for i in range(len(arrList)) if levels[i] <= zoom_level]
    meshList = [meshList[i] for i in range(len(meshList)) if levels[i] <= zoom_level]

matplotlib.rcParams.update({'font.size': 15})

arr_thresh = 0.0
def animate_arr(n):
    plt.clf()
    if plot_mag:
        arr_max = np.max([np.max(np.abs(arr[n])) for arr in arrList])
        if arr_thresh > 0 and arr_max > arr_thresh:
            arr_max = arr_thresh
        if showTotalField:
            arr_max += 1.0
        if showIncidentField:
            arr_max = None
        for ind in range(len(arrList)):
            H, V = meshList[ind]
            field_n = arrList[ind][n]
            if showTotalField:
                field_n += GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], H, V)
            if showIncidentField:
                field_n = GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], H, V)
            if normalizeToSource:
                field_n /= GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], 0, 0)
            plt.pcolormesh(H*lengthScale_si, V*lengthScale_si, np.abs(field_n), 
                           cmap="rainbow", vmax=arr_max)
            
        hLabel, vLabel = axisLabel
        xlabel(hLabel, fontsize="20")
        ylabel(vLabel, fontsize="20")

        title("@ {} ps".format(n*timeScale_si))
        plt.tight_layout()
        
    else:
        arr_max = np.max([np.max(arr[n]) for arr in arrList])
        arr_min = np.min([np.min(arr[n]) for arr in arrList])
        if arr_thresh > 0 and np.abs(arr_max) > arr_thresh:
            arr_max = arr_thresh
        if arr_thresh > 0 and np.abs(arr_min) > arr_thresh:
            arr_min = -arr_thresh
        if showTotalField:
            arr_max += 1.0
            arr_min -= 1.0
        if showIncidentField:
            arr_max = 1.0
            arr_min = -1.0
        for ind in range(len(arrList)):
            H, V = meshList[ind]
            field_n = arrList[ind][n]
            if showTotalField:
                field_n += GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], H, V)
            if showIncidentField:
                field_n = GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], H, V)
            if normalizeToSource:
                field_n /= GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], 0, 0)
            plt.pcolormesh(H*lengthScale_si, V*lengthScale_si, field_n, 
                           cmap="rainbow", vmin=arr_min, vmax=arr_max)

        hLabel, vLabel = axisLabel
        xlabel("{} ({})".format(hLabel, lengthUnit), fontsize="20")
        ylabel("{} ({})".format(vLabel, lengthUnit), fontsize="20")

        title("@ {:.4f} {}".format(n*timeScale_si, timeUnit), fontsize="18")
        plt.tight_layout()

    fig = plt.gcf()
    plt.colorbar()
    plt.pause(0.05)
    return fig

save_animation = False

Dh_m = axisRange[0][1] - axisRange[0][0] 
Dv_m = axisRange[1][1] - axisRange[1][0] 
if save_animation:
    fig = plt.figure(figsize=(7,7*(Dv_m/Dh_m)))
    anim = animation.FuncAnimation(fig, animate_arr, frames=arrList[0].shape[0], interval=1, repeat=False)
    anim.save("../data/3D/auto/" + 'Efield-anim.mp4', writer="ffmpeg", fps=15, dpi=200)
else:    
    plt.ion()
    plt.figure(figsize=(12,12*(Dv_m/Dh_m)))
    for n in range(arrList[0].shape[0]):
        animate_arr(n)

        
#animate_arr(maxPoint_center[0][0])
#plt.savefig("../data/3D/auto/Efield.png", bbox_inches=None, pad_inches=0.4)
#plt.savefig("../data/3D/auto/Efield.eps", bbox_inches=None, pad_inches=0.4)


Populating the interactive namespace from numpy and matplotlib
maxPoint_center:  [(162, 0, 200, 83), 16.591566]


/home/nima/.local/lib/python3.6/site-packages/ipykernel_launcher.py:77: RuntimeWarning: divide by zero encountered in true_divide
/home/nima/.local/lib/python3.6/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


### Animate multiple folders (to run on the server)

In [3]:
import matplotlib
matplotlib.use("Agg")

import numpy as np
from scipy import constants
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import pickle
import os

from ReadBinary import *

def GetAnimations(simfolder, animation_folder):
    def GetViewArrays(view, simfolder=simfolder, max_level=-1):
        arrList = []
        meshList = []
        axisRange = [[], []]
        axisLabel = []
        levels = []
        maxPoint_center = None

        def AdjustAxisMinMax(axisRange, new_min_max):
            ax_min, ax_max = new_min_max
            if axisRange == []:
                axisRange.extend([ax_min, ax_max])
            else:
                if axisRange[0] > ax_min:
                    axisRange[0] = ax_min
                if axisRange[1] < ax_max:
                    axisRange[1] = ax_max

        if view["type"] == "partial":
            for gridName, params in gridParams.items():
                if max_level > 0 and params["level"] > max_level:
                    continue
                r0, r1 = params["r0"], params["r1"]
                plane = {"x":0, "y":1, "z":2}
                if r0[plane[view["plane"]]] <= view["at"] < r1[plane[view["plane"]]]:
                    dataFileName =  simfolder + "/build/data/3D/auto/" + \
                                    gridName + "_" + view["arrayName"] + "_" + view["direction"] + \
                                    "_@" + view["plane"] + "=" + str(view["at"]) + ".data"
                    arr = GetArrays(dataFileName)
                    H, V = None, None
                    hRange, vRange = axisRange
                    axLabel = None
                    level = params["level"]
                    if level == 0:
                        ind_max = np.unravel_index(np.argmax(arr, axis=None), arr.shape)
                        maxPoint_center = [ind_max, arr[ind_max]]
                        print("maxPoint_center: ", maxPoint_center)
                    if view["plane"] == "x":
                        arr = arr[:, 0, :, :]
                        z = np.linspace(r0[2], r1[2], arr.shape[2])
                        y = np.linspace(r0[1], r1[1], arr.shape[1])
                        Z, Y = np.meshgrid(z, y, indexing="xy")
                        H, V = Z, Y
                        AdjustAxisMinMax(hRange, (r0[2], r1[2]))
                        AdjustAxisMinMax(vRange, (r0[1], r1[1]))
                        axLabel = ["z", "y"]

                    elif view["plane"] == "y":
                        arr = arr[:, :, 0, :]
                        z = np.linspace(r0[2], r1[2], arr.shape[2])
                        x = np.linspace(r0[0], r1[0], arr.shape[1])
                        Z, X = np.meshgrid(z, x, indexing="xy")
                        H, V = Z, X
                        AdjustAxisMinMax(hRange, (r0[2], r1[2]))
                        AdjustAxisMinMax(vRange, (r0[0], r1[0]))
                        axLabel = ["z", "x"]

                    elif view["plane"] == "z":
                        arr = arr[:, :, :, 0]
                        y = np.linspace(r0[1], r1[1], arr.shape[2])
                        x = np.linspace(r0[0], r1[0], arr.shape[1])
                        X, Y = np.meshgrid(x, y, indexing="ij")
                        H, V = X, Y
                        AdjustAxisMinMax(hRange, (r0[0], r1[0]))
                        AdjustAxisMinMax(vRange, (r0[1], r1[1]))
                        axLabel = ["x", "y"]

                    else:
                        assert False
                    arrList.append(arr)
                    meshList.append([H, V])
                    axisRange = [hRange, vRange]
                    axisLabel = axLabel
                    levels.append(level)

        return arrList, meshList, axisRange, axisLabel, levels, maxPoint_center


    fileParams = open(os.path.join(simfolder, "build/data/3D/auto/params.param"), "rb")
    params = pickle.load(fileParams)
    fileParams.close()
    #print("params: ", params)

    gridParams = params["grids"]
    viewParams = params["views"]
    unitParams = params["units"]
    boxesParams = params["boxes"]
    sourceParams = params["sources"][0]
    geomParams = params["geometries"][0]

    dt_coarse = boxesParams[-1]["dt"]
    length_FD = unitParams["length_FD"]

    from PhysicalUnits import PhysicalUnits
    units = PhysicalUnits(length_FD)
    dt_SI = units.ConvertFDTimeToSIUnits(dt_coarse)

    tipHeight = geomParams["height"]

    print("views: ")
    for i in range(len(viewParams)):
        print(i, " : ", viewParams[i])

    print(sourceParams)
    A_src = sourceParams["amplitude"]
    v_src = sourceParams["propagationVelocity"]
    t0_src = sourceParams["t_center"]
    w_src = 2.0*np.pi*sourceParams["modulationFrequency"]
    phi_src = sourceParams["modulationPhase"]
    t_width_src = sourceParams["rectWidth"]
    t_edgewidth_src = sourceParams["rectEdgeWidth"]
    def rect(x, width, edge_width):
        return (np.abs(x) <= (width - edge_width)/2) \
               + 0.5 * (np.abs(x + width/2) < edge_width/2) * (1.0 + np.sin((x + width/2)/edge_width * np.pi)) \
               + 0.5 * (np.abs(x - width/2) < edge_width/2) * (1.0 - np.sin((x - width/2)/edge_width * np.pi))

    def GetSourceField(t, plane, H, V):
        if plane == "x":
            Ey_inc = A_src*np.cos(w_src*(t - t0_src - H/v_src) + phi_src) \
                     * rect(t - t0_src - H/v_src, t_width_src, t_edgewidth_src)
            return Ey_inc
        elif plane == "y":
            Ey_inc = A_src*np.cos(w_src*(t - t0_src - H/v_src) + phi_src) \
                     * rect(t - t0_src - H/v_src, t_width_src, t_edgewidth_src)
            return Ey_inc
        elif plane == "z":
            Ey_inc = A_src*np.cos(w_src*(t - t0_src) + phi_src) \
                     * rect(t - t0_src, t_width_src, t_edgewidth_src)
            return Ey_inc


    def AnimateStuff(viewInd, zoom):
        #viewInd = 0
        arrList, meshList, axisRange, axisLabel, levels, maxPoint_center = GetViewArrays(viewParams[viewInd], max_level=6)
        plot_mag = False
        lengthScale_si = length_FD/constants.micro
        lengthUnit = "um"
        timeScale_si = dt_SI/constants.pico
        timeUnit = "ps"
        showTotalField = True
        #zoom = True
        zoom_level = 0

        if zoom:
            arrList = [arrList[i] for i in range(len(arrList)) if levels[i] <= zoom_level]
            meshList = [meshList[i] for i in range(len(meshList)) if levels[i] <= zoom_level]

        matplotlib.rcParams.update({'font.size': 15})

        arr_thresh = 0.0
        def animate_arr(n):
            plt.clf()
            if plot_mag:
                arr_max = np.max([np.max(np.abs(arr[n])) for arr in arrList])
                if arr_thresh > 0 and arr_max > arr_thresh:
                    arr_max = arr_thresh
                if showTotalField:
                    arr_max += 1.0
                for ind in range(len(arrList)):
                    H, V = meshList[ind]
                    field_n = arrList[ind][n]
                    if showTotalField:
                        field_n += GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], H, V)
                    plt.pcolormesh(H*lengthScale_si, V*lengthScale_si, np.abs(field_n), 
                                   cmap="rainbow", vmax=arr_max)

                hLabel, vLabel = axisLabel
                plt.xlabel(hLabel, fontsize="20")
                plt.ylabel(vLabel, fontsize="20")

                plt.title("@ {} ps".format(n*timeScale_si))
                plt.tight_layout()

            else:
                arr_max = np.max([np.max(arr[n]) for arr in arrList])
                arr_min = np.min([np.min(arr[n]) for arr in arrList])
                if arr_thresh > 0 and np.abs(arr_max) > arr_thresh:
                    arr_max = arr_thresh
                if arr_thresh > 0 and np.abs(arr_min) > arr_thresh:
                    arr_min = -arr_thresh
                if showTotalField:
                    arr_max += 1.0
                    arr_min -= 1.0
                for ind in range(len(arrList)):
                    H, V = meshList[ind]
                    field_n = arrList[ind][n]
                    if showTotalField:
                        field_n += GetSourceField(n*dt_coarse, viewParams[viewInd]["plane"], H, V)
                    plt.pcolormesh(H*lengthScale_si, V*lengthScale_si, field_n, 
                                   cmap="rainbow", vmin=arr_min, vmax=arr_max)

                hLabel, vLabel = axisLabel
                plt.xlabel("{} ({})".format(hLabel, lengthUnit), fontsize="20")
                plt.ylabel("{} ({})".format(vLabel, lengthUnit), fontsize="20")

                plt.title("@ {:.4f} {}".format(n*timeScale_si, timeUnit), fontsize="18")
                plt.tight_layout()

            fig = plt.gcf()
            plt.colorbar()
            plt.pause(0.05)
            return fig

        save_animation = True

        outPlaneName = {"x":"yz", "y":"xz", "z":"xy"}
        anim_name = "Ey" + "_@_" + outPlaneName[viewParams[viewInd]["plane"]] + "_plane"
        if zoom:
            anim_name += "_zoom"

        anim_folder = os.path.join(animation_folder, simfolder.replace("/", ""))
        if not os.path.exists(anim_folder):
            os.makedirs(anim_folder)

        Dh_m = axisRange[0][1] - axisRange[0][0] 
        Dv_m = axisRange[1][1] - axisRange[1][0] 
        if save_animation:
            fig = plt.figure(figsize=(7,7*(Dv_m/Dh_m)))
            anim = animation.FuncAnimation(fig, animate_arr, frames=arrList[0].shape[0], interval=1, repeat=False)
            anim.save(os.path.join(anim_folder, anim_name + '.mp4'), writer="ffmpeg", fps=15, dpi=200)
        else:    
            plt.ion()
            plt.figure(figsize=(12,12*(Dv_m/Dh_m)))
            for n in range(arrList[0].shape[0]):
                animate_arr(n)
        
        plt.close('all')
        if zoom:
            animate_arr(maxPoint_center[0][0])
            plt.savefig(os.path.join(anim_folder, anim_name + '.png'), bbox_inches="tight", pad_inches=0.4)
            plt.savefig(os.path.join(anim_folder, anim_name + '.eps'), bbox_inches="tight", pad_inches=0.4)
            plt.close('all')

    for viewInd in range(len(viewParams)):
        AnimateStuff(viewInd, False)
        plt.close('all')
        
        AnimateStuff(viewInd, True)
        plt.close('all')

simfolders = ["../../../YEEFDTD"]
anim_folder = "E_animation"
for simfolder in simfolders:
    GetAnimations(simfolder, anim_folder)
    

views: 
0  :  {'type': 'partial', 'plane': 'x', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
1  :  {'type': 'partial', 'plane': 'y', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
2  :  {'type': 'partial', 'plane': 'z', 'at': 0.0, 'direction': 'y', 'arrayName': 'E'}
{'type': 'PureScatteredRectPlaneWave', 'geometryName': 'tip', 'polarization': 'y', 'amplitude': 1.0, 'propagationDirection': array([0., 0., 1.]), 'propagationVelocity': 1.0, 't_center': 899.3773739999998, 'rectWidth': 1498.9622899999997, 'rectEdgeWidth': 149.89622899999998, 'modulationFrequency': 0.0006671281903963042, 'modulationPhase': 1.5707963267948966}
maxPoint_center:  [(52, 0, 229, 97), 63.992893]
maxPoint_center:  [(52, 0, 229, 97), 63.992893]
maxPoint_center:  [(53, 96, 0, 101), 59.752605]
maxPoint_center:  [(53, 96, 0, 101), 59.752605]
maxPoint_center:  [(52, 97, 229, 0), 63.988472]
maxPoint_center:  [(52, 97, 229, 0), 63.988472]
